In [1]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM
import string
from trl import SFTTrainer, SFTConfig, DataCollatorForCompletionOnlyLM
from peft import LoraConfig

In [2]:
## モデル名

model_name = "cyberagent/calm2-7b-chat"

In [3]:
## トークナイザーのインポート

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast = True
)

In [4]:
## モデルのインポート
## リソースが足りないので、1/4量子化

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map = 'auto', # 複数のGPUがあるときに、それらを均等に使用する
    load_in_8bit = True, # 精度と引き換えに、パラメータを8bitに省略したLLMを使用して計算を軽量化
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
chat_template = string.Template(
    "USER:${user}\nASSISTANT:${assistant}"
)
response_key = "ASSISTANT:"

In [6]:
sample_text = chat_template.safe_substitute({
    "user": "人気タレント・タモリの本名は何でしょう？",
     "assistant": "" # AIに回答させるために空けておく
})

In [7]:
def generate(model, text):
    input_ids = tokenizer.encode(
        text,
        return_tensors = 'pt',
        add_special_tokens = True
    ).to(model.device)
    output_ids = model.generate(
        input_ids,
        max_new_tokens = 100,
        do_sample = True,
        temperature = 0.8,
    )
    print(
        tokenizer.decode(
            output_ids[0],
            skip_special_tokens = True
        )
    )

In [8]:
%%time
generate(model, sample_text)

USER:人気タレント・タモリの本名は何でしょう？
ASSISTANT: タモリの本名は森田豊です。
CPU times: user 1.69 s, sys: 294 ms, total: 1.98 s
Wall time: 1.98 s


In [9]:
## 外部のデータセットをインポート

rs = "izumi-lab/llm-japanese-dataset"
datasets = load_dataset(
    rs,
    split = 'train'
)

In [10]:
## 学習データセットの整形プロセス

def update_datasets(example):
    output_texts = []
    for instruction_, input_, output_ in zip(example['instruction'], example['input'], example['output']):
        text = chat_template.safe_substitute({
            "user": "{0}\n{1}".format(instruction_, input_),
            "assistant": output_
        })
        output_texts.append(text)
    return output_texts

In [11]:
## ??

collator = DataCollatorForCompletionOnlyLM(
    response_key,
    tokenizer = tokenizer
)

In [12]:
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout = 0.1,
    r = 16,
    bias = "lora_only",
    task_type = "CAUSAL_LM",
)

In [13]:
## アダプタの付与
## 量子化したモデルをそのままファインチューニングできないため
## 全体チューニングではなく、出力換算するための行列を修飾する(Lora)
## 実際にtrainされるのは行列の部分だけ

model.add_adapter(peft_config)

In [14]:
%%time

## 訓練オブジェクトの定義

trainer = SFTTrainer(
    model,
    train_dataset = datasets.select(range(10000)),
    args = SFTConfig(
        output_dir = "./tmp",
        per_device_train_batch_size = 16
    ),
    formatting_func = update_datasets,
    data_collator = collator,
    max_seq_length = 128,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

CPU times: user 3.13 s, sys: 106 ms, total: 3.23 s
Wall time: 1.02 s


In [ ]:
%%time

## 訓練実行

trainer.train()

Step,Training Loss


/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `ASSISTANT:` in the following instance: USER:和服で、帯の結び目を丸くふくらませる結び方を、ある楽器に例えて何というでしょう？

ASSISTANT:(お)太鼓結び<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|> This instance will be ignored in loss calculation. Note, if this happens often, consider increasing the `max_seq_length`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/utils.py:160: UserWarning: Could not find response key `ASSISTANT:` in the following instance: USER:嘉永６年に開園した、東京にある日本最古の遊園地はどこでしょう？

ASSISTANT:(浅草)花やしき<|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padding|><|padd

In [ ]:
%%time
generate(model, sample_text)

データ多すぎる説がある

datasets.select(range(1000))とか検討

USER, USER, USER...

この現象はアダプタを付けた時点で発症しているので、
アダプタの設定が悪い可能性がある

要検証